<a href="https://colab.research.google.com/github/techmoksha/iisc-cds-modules/blob/module-1/M2_NB_MiniProject_1_LinearRegression_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Linear Regression with Regularization

## Problem Statement

Predict the bike-sharing counts per hour based on the features including weather, day, time, humidity, wind speed, season e.t.c.

## Learning Objectives

At the end of the mini-project, you will be able to :

* perform data exploration and visualization
* implement linear regression using sklearn and optimization
* apply regularization on regression using Lasso, Ridge and Elasticnet techniques
* calculate and compare the MSE value of each regression technique
* analyze the features that are best contributing to the target

### Dataset

The dataset chosen for this mini-project is [Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset).  This dataset contains the hourly and daily count of rental bikes between the years 2011 and 2012 in the capital bike share system with the corresponding weather and seasonal information. This dataset consists of 17389 instances of 16 features.

Bike sharing systems are a new generation of traditional bike rentals where the whole process from membership, rental and return has become automatic. Through these systems, the user can easily rent a bike from a particular position and return to another position. Currently, there are about over 500 bike-sharing programs around the world which is composed of over 500 thousand bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

Apart from interesting real world applications of bike sharing systems, the characteristics of data being generated by these systems make them attractive for the research. As opposed to other transport services such as bus or subway, the duration of travel, departure and arrival position are explicitly recorded in these systems. This feature turns bike sharing system into a virtual sensor network that can be used for sensing mobility in the city. Hence, it is expected that the most important events in the city could be detected via monitoring these data.

<img src="https://s26551.pcdn.co/wp-content/uploads/2012/02/resize-va-sq-bikeshare.jpg" alt="drawing" width="400"/>

### Data Fields

* dteday - hourly date
* season - 1:winter, 2:spring, 3:summer, 4:fall
* hr - hour
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weathersit -<br>
    1 - Clear, Few clouds, Partly cloudy, Partly cloudy <br>
    2 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist<br>
    3 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds<br>
    4 - Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog<br>   
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* cnt - number of total rentals

## Information

**Regularization:** It is a form of regression that shrinks the coefficient estimates towards zero. In other words, this technique discourages learning a more complex or flexible model, to avoid the risk of overfitting. A simple relation for linear regression looks like this.

$Y ≈ β_0 + β_1 X_1 + β_2 X_2 + …+ β_p X_p$

 Here $Y$ represents the learned relation and $β$ represents the coefficient estimates for different variables or predictors(X).

 If there is noise in the training data, then the estimated coefficients won’t generalize well to the future data. This is where regularization comes in and shrinks or regularizes these learned estimates towards zero.

Below are the Regularization techniques:

 * Ridge Regression
 * Lasso Regression
 * Elasticnet Regression

## Grading = 10 Points

In [ ]:
#@title Download the dataset
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/Bike_Sharing_Dataset.zip
!unzip Bike_Sharing_Dataset.zip

#### Importing Necessary Packages

In [ ]:
# Loading the Required Packages
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Data Loading

In [ ]:
# Read the hour.csv file
# YOUR CODE HERE
df = pd.read_csv('hour.csv')
df

print the first five rows of dataset

In [ ]:
# YOUR CODE HERE
df.head()

print the datatypes of the columns

In [ ]:
# YOUR CODE HERE
df.dtypes

In [ ]:
df['hr'].unique().shape[0]

### Task flow with respect to feature processing and model training

* Explore and analyze the data

* Identify continuous features and categorical features

* Apply scaling on continuous features and one-hot encoding on categorical features

* Separate the features, targets and split the data into train and test

* Find the coefficients of the features using normal equation and find the cost (error)

* Apply batch gradient descent technique and find the best coefficients

* Apply SGD Regressor using sklearn

* Apply linear regression using sklearn

* Apply Lasso, Ridge, Elasticnet Regression

### EDA &  Visualization ( 2 points)

#### Visualize the hour (hr) column with an appropriate plot and find the busy hours of bike sharing

In [ ]:
count_grouped_by_hour = df.groupby('hr')['cnt'].sum().reset_index()
count_grouped_by_hour

In [ ]:
plt.hist?

In [ ]:
count_grouped_by_hour.plot(kind="bar",stacked=True, figsize=(10, 8))

plt.xlabel("Hour")
plt.ylabel("Number of Bikers")
plt.title("Distribution of Bikers every Hour across all days")
plt.xticks(rotation=0, ha='right')
plt.spring()
plt.show()

It appears that morning 8 AM and evening 5, 6 PM are busiest


#### Visualize the distribution of count, casual and registered variables

In [ ]:
# YOUR CODE HERE for distribuiton of count variable
plt.hist(df['cnt'], bins=50, edgecolor='black')

# Add labels and title
plt.title('Histogram of Counts')
plt.xlabel('Value')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:
# YOUR CODE HERE for distribuiton of casual variable
plt.hist(df['casual'], bins=50, edgecolor='black')

# Add labels and title
plt.title('Histogram of Casual Users')
plt.xlabel('Value')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:
# YOUR CODE HERE for distribuiton of registered variable
plt.hist(df['registered'], bins=50, edgecolor='black')

# Add labels and title
plt.title('Histogram of Registered Users')
plt.xlabel('Value')
plt.ylabel('Frequency')

# Show the plot
plt.show()

#### Describe the relation of weekday, holiday and working day

In [ ]:
df.sample(20)

In [ ]:
# YOUR CODE HERE
correlation_matrix = df[['weekday', 'holiday', 'workingday']].corr(method='pearson')
correlation_matrix

In [ ]:
spearman_corr = df['holiday'].corr(df['workingday'], method='spearman')
spearman_corr

#### Visualize the month wise count of both casual and registered for the year 2011 and 2012 separately.

Hint: Stacked barchart

In [ ]:
# stacked bar chart for year 2011
# YOUR CODE HERE
casual_registered_per_year = df.groupby(['yr', 'mnth']).agg(
    total_casual=('casual', 'sum'),
    total_registered=('registered', 'sum')
).reset_index()
casual_registered_per_year

In [ ]:
months = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12')

In [ ]:
def stacked_bar_plot(yr: int, casual_users, registered_users):
    ind = np.arange(12)    # the 12 months
    width = 0.4
    p1 = plt.bar(ind, casual_users, width)
    p2 = plt.bar(ind, registered_users, width,
                bottom=casual_users)
    plt.xlabel(f"Months of {yr}")
    plt.ylabel("Number of Users")
    plt.title(f"Distribution of Casual and Registered Users across 12 months of {yr}")
    plt.xticks(ind, months, rotation=90, ha='right')
    plt.legend((p1[0], p2[0]), ('Casual', 'Registered'))
    plt.spring()
    plt.show()

In [ ]:
total_casual_yr_2011 = casual_registered_per_year.query('yr == 0')['total_casual']
total_registered_yr_2011 = casual_registered_per_year.query('yr == 0')['total_registered']
stacked_bar_plot('2011', total_casual_yr_2011, total_registered_yr_2011)

In [ ]:
# stacked bar chart for year 2012
# YOUR CODE HERE
total_casual_yr_2012 = casual_registered_per_year.query('yr == 1')['total_casual']
total_registered_yr_2012 = casual_registered_per_year.query('yr == 1')['total_registered']
stacked_bar_plot('2012', total_casual_yr_2012, total_registered_yr_2012)

#### Analyze the correlation between features with heatmap

In [ ]:
# YOUR CODE HERE
df_new = df.drop('dteday', axis=1)
df_new = df_new.apply(pd.to_numeric, errors='coerce')
plt.figure(figsize=(14, 14))
sns.heatmap(df_new.corr(), annot=True, linewidth=0.5, center=0)
plt.show()

It appears that temp and atemp have high correlation. Similarly season and month have high correlations too.

#### Visualize the box plot of casual and registered variables to check the outliers

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(10, 6))  # Adjust figure size if needed

# Box plot for 'casual'
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
sns.boxplot(y=df_new['casual'])
plt.title('Box Plot of Casual')
plt.ylabel('Casual')

# Box plot for 'registered'
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
sns.boxplot(y=df_new['registered'])
plt.title('Box Plot of Registered')
plt.ylabel('Registered')

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

**Casual Median** : 20
**Casual Max** : 120

**Registered Median** : 120
**Registered Max** : ~500

Value spread higher for Registered, whereas Outlier Spread higher for casual.


### Pre-processing and Data Engineering (1 point)

#### Drop unwanted columns

In [ ]:
# YOUR CODE HERE
# Dropping atemp and windspeed as they do not seem to have any impact on bike sharing
df_new = df_new.drop(['atemp', 'windspeed'], axis=1)
df_new

#### Identify categorical and continuous variables


In [ ]:
# YOUR CODE HERE
max_unique_for_categorical = 24

categorical_columns = []
continuous_columns = []

[categorical_columns.append(col) if df_new[col].dtype == 'object' or  df[col].nunique() <= max_unique_for_categorical else continuous_columns.append(col) for col in df_new.columns]

print("Categorical variables:", categorical_columns)
print("Continuous variables:", continuous_columns)

#### Feature scaling

Feature scaling is essential for machine learning algorithms, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance. Apply scaling on the continuous variables on the given data.

Hint: `MinMaxScaler` or `StandardScaler`



In [ ]:
# YOUR CODE HERE
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_new[continuous_columns]), columns=continuous_columns)
df_scaled

#### Apply one-hot encode on the categorical data

One-hot encoding is applied on the categorical variables, which should not have a different weight or order attached to them, it is presumed that all categorical variables have equivalent "values". This means that you cannot simply order them from zero to the number of categories as this would imply that the earlier categories have less "value" than later categories.

Hint: `sklearn.preprocessing.OneHotEncoder`

In [ ]:
# YOUR CODE HERE
encoder = OneHotEncoder(sparse_output=False)

df_one_hot_encoded = encoder.fit_transform(df_new[categorical_columns])

# Convert the array to a DataFrame and add column names
df_encoded = pd.DataFrame(df_one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
# Combine categorical and continuous
df_encoded = pd.concat([df_scaled, df_encoded], axis=1)

print(f"Encoded Employee data : \n{df_encoded}")

In [ ]:
encoder.get_feature_names_out?

In [ ]:
df_encoded.columns

#### Specify features and targets after applying scaling and one-hot encoding

In [ ]:
from sklearn.impute import SimpleImputer
# YOUR CODE HERE
pred_features_columns = ['hr_0', 'hr_1', 'hr_2', 'hr_3', 'hr_4',
       'hr_5', 'hr_6', 'hr_7', 'hr_8', 'hr_9', 'hr_10', 'hr_11', 'hr_12',
       'hr_13', 'hr_14', 'hr_15', 'hr_16', 'hr_17', 'hr_18', 'hr_19', 'hr_20',
       'hr_21', 'hr_22', 'hr_23', 'holiday_0', 'holiday_1', 'hum', 'temp', 'season_1', 'season_2','season_3','season_4','workingday_0','workingday_1','weathersit_1','weathersit_2','weathersit_3','weathersit_4','weekday_0','weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6']

X = df_encoded[pred_features_columns]
# Imputing missing values in X
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
X = pd.DataFrame(X, columns=pred_features_columns)
# Target features is total count, casual and registered users of bikes
y = df_encoded[['cnt', 'casual', 'registered']]
# YOUR CODE HERE to show 'X'
X

In [ ]:
print(f"Number of features selected for prediction {len(pred_features_columns)}")

### Implement the linear regression by finding the coefficients using below approaches (2 points)

* Find the coefficients using normal equation

* (Optional) Implement batch gradient descent

* (Optional) SGD Regressor from sklearn

#### Select the features and target and split the dataset

As there are 3 target variables, choose the count (`cnt`) variable.

In [ ]:
# YOUR CODE HERE
y = df_encoded['cnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state= 101)

In [ ]:
y_train.head()

#### Implementation using Normal Equation

$\theta = (X^T X)^{-1} . (X^T Y)$

$θ$ is the hypothesis parameter that defines the coefficients

$X$ is the input feature value of each instance

$Y$ is Output value of each instance

For performing Linear Regression Using the Normal Equation refer [here](https://cdn.iisc.talentsprint.com/CDS/Assignments/Module2/M2_SNB_MiniProject_1_LinearRegression_Regularization_Performing%20Linear%20Regression%20using%20Normal%20equation.pdf).

To solve the normal equation compute least-squares solution by using `scipy.linalg`

Hint: [scipy.linalg.lstsq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html)

In [ ]:
# YOUR CODE HERE
X = np.array(X_train)
Y = np.array(y_train)
# Using least squares on training data set.
theta_vec, residuals, rank, s = np.linalg.lstsq(X, Y, rcond=None)

In [ ]:
feature_to_coefficients = {X_train.columns[i]:theta_vec[i] for i in range(len(X_train.columns))}
feature_to_coefficients

#### (Optional) Implementing Linear regression using batch gradient descent

Initialize the random coefficients and optimize the coefficients in the iterative process by calculating cost and finding the gradient.

Hint: [gradient descent](https://cdn.iisc.talentsprint.com/CDS/Assignments/Module2/M2_SNB_MiniProject_1_LinearRegression_Regularization_Multivariate%20Linear%20Regression.pdf)

In [ ]:
# YOUR CODE HERE
def cost_function(X, Y, theta):
    m = len(Y)
    J = np.sum((X.dot(theta) - Y) ** 2) / 2 * m
    return J

In [ ]:
def batch_gradient_descent(X, Y, theta, alpha, iterations):
    cost_history = [0] * iterations
    m = len(Y)

    for iter in range(iterations):
        # Calculate hypothesis values
        h = X.dot(theta)
        # Difference between actual and calculated
        loss = h - Y
        # Gradient Calculation
        gradient = X.T.dot(loss) / m
        theta -= alpha * gradient
        cost = cost_function(X, Y, theta)
        cost_history[iter] = cost
    return theta, cost_history


In [ ]:
errs_1 = []
theta = np.zeros(X_train.shape[1])         # initial guess
alpha = 0.05                               # learning rate
iterations = 2000

final_theta, cost_history = batch_gradient_descent(X_train, y_train, theta, alpha, iterations)
print(f"Theta values final {final_theta}")
print(f"Cost History {cost_history}")

#### (Optional) SGD Regressor

Scikit-learn API provides the SGDRegressor class to implement SGD method for regression problems. The SGD regressor applies regularized linear model with SGD learning to build an estimator. A regularizer is a penalty (L1, L2, or Elastic Net) added to the loss function to shrink the model parameters.

* Import SGDRegressor from sklearn and fit the data

* Predict the test data and find the error

Hint: [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

In [ ]:
from sklearn.linear_model import SGDRegressor
# YOUR CODE HERE
sgd = SGDRegressor(max_iter=2000, tol=1e-3, penalty='l2', random_state=42)

# Fit the model on the training data
sgd.fit(X_train, y_train)

# Make predictions on the test set
y_pred = sgd.predict(X_test)

# Calculate the mean squared error of the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Intercept:", sgd.intercept_)

In [ ]:
sgd_features_to_coefficients = {X_train.columns[i]:theta_vec[i] for i in range(len(X_train.columns))}
sgd_features_to_coefficients

### Linear regression using sklearn (3 points)

Implement the linear regression model using sklearn

* Import Linear Regression and fit the train data

* Predict the test data and find the error

Hint: [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [ ]:
# YOUR CODE HERE
lr = LinearRegression()
# Fit lin reg model to train
lr.fit(X_train, y_train)

# Predict on test data
pred = lr.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squared Error:', mean_squared_error(y_test, pred))
print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

#### Calculate the $R^2$ (coefficient of determination) of the actual and predicted data

In [ ]:
# YOUR CODE HERE
print('Coefficient of Determination:', r2_score(y_test, pred))

In [ ]:
lr.

#### Summarize the importance of features

Prediction is the weighted sum of the input values e.g. linear regression. Regularization, such as ridge regression and the elastic net, find a set of coefficients to use in the weighted sum to make a prediction. These coefficients can be used directly as a crude type of feature importance score.
This assumes that the input variables have the same scale or have been scaled prior to fitting a model.

Use the coefficients obtained through the sklearn Linear Regression implementation and create a bar chart of the coefficients.

In [ ]:
df_coefficients = pd.DataFrame({'Features': X_train.columns, 'Coefficients': lr.coef_})
df_coefficients

In [ ]:
# YOUR CODE HERE
num_features = np.arange(df_coefficients.shape[0])
df_coefficients.plot(kind="bar",stacked=True, figsize=(14, 8))

plt.xlabel("Features")
plt.ylabel("Coefficients")
plt.title("Distribution of Bikers every Hour across all days")
plt.xticks(num_features, df_coefficients['Features'], rotation=90, ha='right')
plt.spring()
plt.show()


### Regularization methods (2 points)

#### Apply Lasso regression

* Apply Lasso regression with different alpha values given below and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

Hint: [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

In [ ]:
# setting up alpha
alphas = [0.0001, 0.001,0.01, 0.1, 1, 10, 100]

In [ ]:
# YOUR CODE HERE
from sklearn.linear_model import Lasso

coefs = []
least_error_value = -1
alpha_for_least_error = tuple()
# Loop over each alpha and fit a Lasso model
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    pred = lasso.predict(X_test)
    error = mean_squared_error(y_test, pred)

    if least_error_value < 0:
        least_error_value = error
        alpha_for_least_error = (alpha, least_error_value)
    else:
        if error < least_error_value:
            least_error_value = error
            alpha_for_least_error = (alpha, least_error_value)


print(f'Least Mean Squared Error: {alpha_for_least_error[1]} observed for alpha {alpha_for_least_error[0]}')

#### Apply Ridge regression

* Apply Ridge regression with different alpha values given and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

Hint: [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [ ]:
# YOUR CODE HERE
for alpha in alphas:
    rdg = linear_model.Ridge(alpha = alpha)        # instantiate Ridge regressor
    # YOUR CODE HERE to fit(x, y) on 'rdg'
    rdg.fit(X_train, y_train)
    pred = rdg.predict(X_test)
    error = mean_squared_error(y_test, pred)

    if least_error_value < 0:
        least_error_value = error
        alpha_for_least_error = (alpha, least_error_value)
    else:
        if error < least_error_value:
            least_error_value = error
            alpha_for_least_error = (alpha, least_error_value)


print(f'Least Mean Squared Error: {alpha_for_least_error[1]} observed for alpha {alpha_for_least_error[0]}')

#### Apply Elasticnet regression

* Apply Elasticnet regression with different alpha values given and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

Hint: [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)

In [ ]:
# YOUR CODE HERE
for alpha in alphas:
    en = linear_model.ElasticNet(alpha = alpha, random_state = 0, max_iter=1000)        # instantiate ElasticNet regressor
    # YOUR CODE HERE to fit(x, y) on 'en'
    en.fit(X_train, y_train)
    pred = en.predict(X_test)
    error = mean_squared_error(y_test, pred)

    if least_error_value < 0:
        least_error_value = error
        alpha_for_least_error = (alpha, least_error_value)
    else:
        if error < least_error_value:
            least_error_value = error
            alpha_for_least_error = (alpha, least_error_value)


print(f'Least Mean Squared Error: {alpha_for_least_error[1]} observed for alpha {alpha_for_least_error[0]}')

### Determine if there is a reduction in error if two target variables are considered

Consider (`Casual, Registered`) as target and find the error by implementing Linear Regression model from sklearn

### Report Analysis

* Describe your interpretation of the methods that are used to implement linear regression covered in this mini project.
* Comment on performance of the algorithms/methods used.
* Comment about the nature of the data and fitment of linear regression for this data.
* Can you perform a non linear curve fitting using linear regression? If yes, How?
